### 📄 Colab Code: Real-Time Moon Phase Using Skyfield


In [ ]:
# 📦 Install Skyfield (only needed once)
!pip install skyfield pytz

In [ ]:
# ✅ COLAB-Compatible Skyfield Moon Phase Debug Script

from skyfield.api import load, wgs84
from skyfield.almanac import phase_angle
from datetime import datetime
import pytz  # For timezone-aware conversion

# 🌍 Observer's location (Islamabad, UTC+5)
latitude, longitude = 33.6844, 73.0479

# Load timescale and ephemeris
ts = load.timescale()
eph = load('de421.bsp')
earth, moon, sun = eph['earth'], eph['moon'], eph['sun']

# Force consistent UTC time in Colab
utc_now = datetime.utcnow().replace(tzinfo=pytz.utc)
local_tz = pytz.timezone('Asia/Karachi')
local_dt = utc_now.astimezone(local_tz)
t = ts.from_datetime(utc_now)

print("🕒 Local Time:", local_dt.strftime("%Y-%m-%d %H:%M:%S"))
print("🕒 UTC Time  :", t.utc_iso())

# Observer location
observer = earth + wgs84.latlon(latitude, longitude)

# Apparent positions of moon and sun
astrometric_moon = observer.at(t).observe(moon).apparent()
astrometric_sun = observer.at(t).observe(sun).apparent()

# Moon's altitude & azimuth
alt, az, _ = astrometric_moon.altaz()
print(f"🌙 Moon Altitude: {alt.degrees:.2f}°, Azimuth: {az.degrees:.2f}°")

# ✅ Correct usage of phase_angle (takes 3 args: eph, moon, sun)
try:
    angle = astrometric_moon.separation_from(astrometric_sun)
    print(f"🌓 Phase Angle: {angle.degrees:.2f}°")
    moon_phase = 1 - abs((angle.degrees - 180) / 180)  # 0=new, 1=full
    print(f"🌕 Moon Phase (0=new, 1=full): {moon_phase:.2f}")
except Exception as e:
    print("❌ Error computing moon phase:", e)

# Debugging raw positions (optional)
print("🔭 Moon position (AU):", astrometric_moon.position.au)
print("☀️ Sun position  (AU):", astrometric_sun.position.au)


🕒 Local Time: 2025-07-12 23:37:36
🕒 UTC Time  : 2025-07-12T18:37:36Z
🌙 Moon Altitude: 26.92°, Azimuth: 143.63°
🌓 Phase Angle: 156.34°
🌕 Moon Phase (0=new, 1=full): 0.87
🔭 Moon position (AU): [ 0.00175466 -0.00161887 -0.00088635]
☀️ Sun position  (AU): [-0.35273436  0.87471533  0.3791354 ]


In [ ]:
# 🌍🌞 Compare Astronomical Positions: Earth vs. Sun Center

from skyfield.api import load

ts = load.timescale()
t = ts.now()
eph = load('de421.bsp')

body_map = {
    1:  'Mercury Barycenter',
    2:  'Venus Barycenter',
    3:  'Earth Barycenter',
    4:  'Mars Barycenter',
    5:  'Jupiter Barycenter',
    6:  'Saturn Barycenter',
    7:  'Uranus Barycenter',
    8:  'Neptune Barycenter',
    9:  'Pluto Barycenter',
    10: 'Sun',
    199:'Mercury',
    299:'Venus',
    399:'Earth',
    301:'Moon',
    499:'Mars'
}

earth = eph['earth']
print("📅 UTC Time:", t.utc_iso())
print("\n🪐 Real-Time Positions (in AU): Earth-Centered vs SSB (absolute)\n")
print(f"{'Body':<24} | {'X_Earth':>11} {'Y_Earth':>11} {'Z_Earth':>11} | {'X_SSB':>11} {'Y_SSB':>11} {'Z_SSB':>11}")

for key, name in body_map.items():
    try:
        target = eph[key]

        # Earth-relative position
        geo = earth.at(t).observe(target).apparent().position.au

        # Absolute position from SSB
        abs_pos = target.at(t).position.au

        print(f"{name:<24} | {geo[0]:>11.6f} {geo[1]:>11.6f} {geo[2]:>11.6f} | {abs_pos[0]:>11.6f} {abs_pos[1]:>11.6f} {abs_pos[2]:>11.6f}")

    except Exception as e:
        print(f"{name:<24} | Error: {e}")



📅 UTC Time: 2025-07-12T18:37:53Z

🪐 Real-Time Positions (in AU): Earth-Centered vs SSB (absolute)

Body                     |     X_Earth     Y_Earth     Z_Earth |       X_SSB       Y_SSB       Z_SSB
Mercury Barycenter       |   -0.492725    0.476512    0.180959 |   -0.144190   -0.403222   -0.200232
Venus Barycenter         |    0.372594    0.892609    0.341340 |    0.721018    0.013073   -0.039756
Earth Barycenter         |    0.000021   -0.000020   -0.000010 |    0.348516   -0.879691   -0.381164
Mars Barycenter          |   -1.929218    0.485091    0.242985 |   -1.580708   -0.394811   -0.138275
Jupiter Barycenter       |   -0.760854    5.589028    2.409803 |   -0.413210    4.709280    2.028622
Saturn Barycenter        |    9.186057    0.378153   -0.236626 |    9.534564   -0.501498   -0.617814
Uranus Barycenter        |   10.113116   16.021927    6.865100 |   10.460137   15.143038    6.484286
Neptune Barycenter       |   29.528476    1.125728   -0.261874 |   29.876995    0.245736   -0

In [ ]:
from skyfield.api import load
import plotly.graph_objects as go

# Load time and ephemeris
ts = load.timescale()
t = ts.now()
eph = load('de421.bsp')
center = eph['sun']

# Planets and moon
bodies = {
    'Mercury': eph['mercury'],
    'Venus': eph['venus'],
    'Earth': eph['earth'],
    'Moon': eph['moon'],
    'Mars': eph['mars'],
    'Jupiter': eph['jupiter barycenter'],
    'Saturn': eph['saturn barycenter'],
    'Uranus': eph['uranus barycenter'],
    'Neptune': eph['neptune barycenter'],
    'Pluto': eph['pluto barycenter']
}

# Get positions from the Sun's perspective
positions = {}
for name, target in bodies.items():
    astrometric = center.at(t).observe(target).apparent()
    positions[name] = astrometric.position.au

# Initialize figure
fig = go.Figure()

# Plot bodies
for name, (x, y, z) in positions.items():
    fig.add_trace(go.Scatter3d(
        x=[x], y=[y], z=[z],
        mode='markers+text',
        text=[name],
        textposition='top center',
        marker=dict(size=6, color='skyblue'),
        name=name
    ))

# Plot the Sun
fig.add_trace(go.Scatter3d(
    x=[0], y=[0], z=[0],
    mode='markers+text',
    text=["Sun"],
    textposition='top center',
    marker=dict(size=10, color='gold'),
    name='Sun'
))

# Update layout for fullscreen UI
fig.update_layout(
    title=dict(
        text=f"🌌 Real-Time Solar System (UTC: {t.utc_iso()})",
        x=0.5,
        font=dict(size=22, color='white')
    ),
    scene=dict(
        xaxis_title='X (AU)',
        yaxis_title='Y (AU)',
        zaxis_title='Z (AU)',
        xaxis=dict(backgroundcolor="black", color="white", gridcolor="gray"),
        yaxis=dict(backgroundcolor="black", color="white", gridcolor="gray"),
        zaxis=dict(backgroundcolor="black", color="white", gridcolor="gray"),
        aspectmode='data',
        bgcolor='black'
    ),
    margin=dict(l=0, r=0, b=0, t=50),
    paper_bgcolor='black',
    plot_bgcolor='black',
    showlegend=False
)

# Fullscreen and responsive mode
fig.show(config={"displayModeBar": True, "displaylogo": False, "responsive": True, "modeBarButtonsToAdd": ["toggleSpikelines", "resetCamera", "toImage", "zoomIn3d", "zoomOut3d"]})


### Hijri Caendar using Real-Time Moon Phase (Option 1)

In [1]:
!pip install skyfield numpy pytz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 367.0/367.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.5/234.5 kB 12.1 MB/s eta 0:00:00


In [21]:
# ✅ Real-Time Hijri Calendar (Step 1-3): New Moon, Sunset, and Visibility
from skyfield.api import load, wgs84, utc
from skyfield.almanac import find_discrete, moon_phases, sunrise_sunset
from datetime import datetime, timedelta
import pytz
import requests

# ========== 🌍 CONFIGURATION ==========
latitude, longitude = 33.6844, 73.0479  # Example: Islamabad
local_tz = pytz.timezone('Asia/Karachi')

# 🌄 Optional: Get elevation using Open-Elevation API
def get_elevation(lat, lon):
    url = "https://api.open-elevation.com/api/v1/lookup"
    response = requests.get(url, params={"locations": f"{lat},{lon}"})
    if response.ok:
        return response.json()['results'][0]['elevation']
    else:
        raise Exception("Failed to fetch elevation")

elevation_m = get_elevation(latitude, longitude)
print(f"📍 Elevation at ({latitude}, {longitude}) = {elevation_m} meters")

# ========== 🔭 SKYFIELD SETUP ==========
# eph = load('de421.bsp')   # Suports dates from  July 29, 1899 to October 9, 2053
eph = load('de440.bsp')  # Suports dates from 1550 CE to 2650 CE
ts = load.timescale()
earth = eph['earth']
sun = eph['sun']
moon = eph['moon']
topos = wgs84.latlon(latitude, longitude, elevation_m)
observer = earth + topos
utc_now = datetime.utcnow().replace(tzinfo=utc)

# ========== 🌑 STEP 1: New Moon Detection ==========
t0 = ts.utc(utc_now - timedelta(days=30))
t1 = ts.utc(utc_now + timedelta(days=30))
times, phases = find_discrete(t0, t1, moon_phases(eph))
new_moons = [t for t, p in zip(times, phases) if p == 0]

print("\n🌑 New Moon Events:")
for t in new_moons:
    utc_dt = t.utc_datetime().replace(tzinfo=utc)
    local_dt = utc_dt.astimezone(local_tz)
    label = "⬅️ Previous" if utc_dt < utc_now else "➡️ Next"
    print(f"• {label} New Moon: {local_dt.strftime('%Y-%m-%d %H:%M %Z')}")

# 📌 Last New Moon before now
last_new_moon = max(t for t in new_moons if t.utc_datetime() <= utc_now)
nm_local = last_new_moon.utc_datetime().replace(tzinfo=utc).astimezone(local_tz)
print(f"\n📅 Last New Moon (Local): {nm_local.strftime('%Y-%m-%d %H:%M %Z')}")

# ========== 🌇 STEP 2: Accurate Sunset Time ==========
date = nm_local.date()
sun_func = sunrise_sunset(eph, topos)
t0_day = ts.utc(date.year, date.month, date.day, 0)
t1_day = ts.utc(date.year, date.month, date.day, 23, 59)

sun_times, is_rising = find_discrete(t0_day, t1_day, sun_func)
sunset_time = next((t for t, rise in zip(sun_times, is_rising) if not rise), None)

if sunset_time is not None:
    sunset_utc = sunset_time.utc_datetime().replace(tzinfo=utc)
    sunset_local = sunset_utc.astimezone(local_tz)
    print(f"🌇 Sunset on New Moon Date: {sunset_local.strftime('%Y-%m-%d %H:%M:%S %Z')}")

    # ========== 🌙 STEP 3: Crescent Visibility Estimation ==========
    t_vis = ts.utc(sunset_utc + timedelta(minutes=5))  # 5 minutes after sunset
    moon_apparent = observer.at(t_vis).observe(moon).apparent()
    sun_apparent = observer.at(t_vis).observe(sun).apparent()

    moon_alt, moon_az, _ = moon_apparent.altaz()
    sun_alt, _, _ = sun_apparent.altaz()
    elongation = moon_apparent.separation_from(sun_apparent)

    print(f"\n📡 Moon Altitude: {moon_alt.degrees:.2f}°")
    print(f"☀️ Sun Altitude : {sun_alt.degrees:.2f}°")
    print(f"📏 Elongation   : {elongation.degrees:.2f}°")

    # Visibility Rule of Thumb
    visible = moon_alt.degrees > 5 and elongation.degrees > 8 and sun_alt.degrees < -5
    verdict = "🌙 Likely Visible" if visible else "🚫 Not Likely Visible"
    print(f"\n🧭 Visibility Verdict: {verdict}")

else:
    print("❌ Sunset time could not be determined.")


📍 Elevation at (33.6844, 73.0479) = 524.0 meters

🌑 New Moon Events:
• ⬅️ Previous New Moon: 2025-06-25 15:31 PKT
• ➡️ Next New Moon: 2025-07-25 00:11 PKT

📅 Last New Moon (Local): 2025-06-25 15:31 PKT
🌇 Sunset on New Moon Date: 2025-06-25 19:22:03 PKT

📡 Moon Altitude: 2.08°
☀️ Sun Altitude : -1.74°
📏 Elongation   : 4.46°

🧭 Visibility Verdict: 🚫 Not Likely Visible


In [36]:
from skyfield.api import load, wgs84, utc
from skyfield.almanac import find_discrete, moon_phases, sunrise_sunset
from datetime import datetime, timedelta
import pytz
import requests

# ========== LOAD ==========
eph = load('de440.bsp')
ts = load.timescale()
sun, moon, earth = eph['sun'], eph['moon'], eph['earth']

# ========== HELPERS ==========
def get_city_data():
    return {
        'name': 'Islamabad',
        'lat': 33.6844,
        'lon': 73.0479,
        'tz': 'Asia/Karachi'
    }

def get_elevation(lat, lon):
    url = "https://api.open-elevation.com/api/v1/lookup"
    response = requests.get(url, params={"locations": f"{lat},{lon}"})
    if response.ok:
        return response.json()['results'][0]['elevation']
    return 0

def get_hijri_today(city):
    now = datetime.now(pytz.timezone(city['tz']))
    today = now.date()
    url = f"http://api.aladhan.com/v1/gToH?date={today.strftime('%d-%m-%Y')}"
    res = requests.get(url)
    data = res.json()['data']['hijri']
    return int(data['day']), int(data['month']['number']), int(data['year']), now

def get_hijri_date_on(city, date):
    url = f"http://api.aladhan.com/v1/gToH?date={date.strftime('%d-%m-%Y')}"
    res = requests.get(url)
    data = res.json()['data']['hijri']
    return int(data['day']), int(data['month']['number']), int(data['year']), data['date']['gregorian']

def get_last_new_moon(date):
    t0 = ts.utc(date - timedelta(days=30))
    t1 = ts.utc(date + timedelta(days=1))
    times, phases = find_discrete(t0, t1, moon_phases(eph))
    new_moons = [t for t, p in zip(times, phases) if p == 0]
    return max(new_moons)

def check_visibility(city, dt):
    elevation = get_elevation(city['lat'], city['lon'])
    topos = wgs84.latlon(city['lat'], city['lon'], elevation)
    observer = eph['earth'] + topos

    dt_datetime = datetime.combine(dt, datetime.min.time()).replace(tzinfo=utc)
    t0 = ts.utc(dt_datetime - timedelta(hours=12))
    t1 = ts.utc(dt_datetime + timedelta(hours=12))

    f = sunrise_sunset(eph, topos)
    times, events = find_discrete(t0, t1, f)
    sunset = next((t for t, e in zip(times, events) if not e), None)

    if sunset is None:
        print("❌ Sunset time could not be determined.")
        return False, None

    t_obs = ts.utc(sunset.utc_datetime() + timedelta(minutes=5))
    moon_ap = observer.at(t_obs).observe(moon).apparent()
    sun_ap = observer.at(t_obs).observe(sun).apparent()

    m_alt, m_az, _ = moon_ap.altaz()
    s_alt, _, _ = sun_ap.altaz()
    elongation = moon_ap.separation_from(sun_ap)

    print(f"\n📊 Crescent Check at {t_obs.utc_datetime().strftime('%Y-%m-%d %H:%M UTC')}")
    print(f"☀️ Sun Altitude: {s_alt.degrees:.2f}°")
    print(f"🌙 Moon Altitude: {m_alt.degrees:.2f}°")
    print(f"📏 Elongation  : {elongation.degrees:.2f}°")

    visible = m_alt.degrees > 5 and elongation.degrees > 8 and s_alt.degrees < -5
    return visible, sunset.utc_datetime().date()

# ========== MAIN ==========
def estimate_local_hijri(city):
    hijri_day, hijri_month, hijri_year, now = get_hijri_today(city)
    today_date = now.date()
    first_date = today_date - timedelta(days=hijri_day - 1)

    print(f"\n📍 {city['name']} (TZ: {city['tz']})")
    print(f"🗓️ Gregorian Now: {now.strftime('%Y-%m-%d %H:%M:%S')} {city['tz']}")
    print(f"🧮 1st of this Hijri month (from API): {first_date}")

    new_moon = get_last_new_moon(first_date)
    visible, crescent_date = check_visibility(city, new_moon.utc_datetime().date())
    hijri_start_date = crescent_date + timedelta(days=1) if visible else crescent_date + timedelta(days=2)

    predicted_date = hijri_start_date + timedelta(days=hijri_day - 1)
    hijri_predicted_day = (today_date - hijri_start_date).days + 1 if today_date >= hijri_start_date else None

    print(f"🌑 New Moon: {new_moon.utc_datetime().date()} | Visible? {'✅' if visible else '❌'}")
    print(f"🌙 First Crescent Seen (Expected): {crescent_date} → 1st Hijri (Predicted): {hijri_start_date}")

    print(f"📅 Predicted Hijri Start Date: {hijri_start_date}")
    print(f"📅 API Hijri Start Date      : {first_date}")
    print(f"📅 Real-Time Gregorian Now   : {now.strftime('%Y-%m-%d %H:%M:%S')} {city['tz']}")
    if hijri_predicted_day:
        print(f"📅 Hijri Today (Predicted)    : {hijri_predicted_day} {hijri_month}/{hijri_year}")
    else:
        print("📅 Hijri Today (Predicted)    : ❌ Date before crescent visibility")
    print(f"📅 Hijri Today (from API)     : {hijri_day} {hijri_month}/{hijri_year}")

# ========== RUN ==========
city = get_city_data()
estimate_local_hijri(city)



📍 Islamabad (TZ: Asia/Karachi)
🗓️ Gregorian Now: 2025-07-14 11:22:13 Asia/Karachi
🧮 1st of this Hijri month (from API): 2025-06-26

📊 Crescent Check at 2025-06-24 14:26 UTC
☀️ Sun Altitude: -1.74°
🌙 Moon Altitude: -8.25°
📏 Elongation  : 12.95°
🌑 New Moon: 2025-06-25 | Visible? ❌
🌙 First Crescent Seen (Expected): 2025-06-24 → 1st Hijri (Predicted): 2025-06-26
📅 Predicted Hijri Start Date: 2025-06-26
📅 API Hijri Start Date      : 2025-06-26
📅 Real-Time Gregorian Now   : 2025-07-14 11:22:13 Asia/Karachi
📅 Hijri Today (Predicted)    : 19 1/1447
📅 Hijri Today (from API)     : 19 1/1447


In [49]:
# ========== INSTALL DEPENDENCIES IF NEEDED ==========
# !pip install skyfield numpy pytz requests

from skyfield.api import load, wgs84, utc
from skyfield.almanac import find_discrete, moon_phases, sunrise_sunset
from datetime import datetime, timedelta
import pytz
import requests

# ========== LOAD ==========
eph = load('de440.bsp')
ts = load.timescale()
sun, moon, earth = eph['sun'], eph['moon'], eph['earth']

# ========== HELPERS ==========
def get_city_data():
    return {
        'name': 'Islamabad',
        'lat': 33.6844,
        'lon': 73.0479,
        'tz': 'Asia/Karachi'
    }

def get_elevation(lat, lon):
    url = "https://api.open-elevation.com/api/v1/lookup"
    response = requests.get(url, params={"locations": f"{lat},{lon}"})
    if response.ok:
        elevation = response.json()['results'][0]['elevation']
        print(f"DEBUG: Elevation for ({lat}, {lon}) → {elevation}m")
        return elevation
    return 0

def get_hijri_today(city):
    now = datetime.now(pytz.timezone(city['tz']))
    print(f"DEBUG: Local datetime now → {now}")
    today = now.date()
    url = f"http://api.aladhan.com/v1/gToH?date={today.strftime('%d-%m-%Y')}"
    res = requests.get(url)
    data = res.json()['data']['hijri']
    day = int(data['day'])
    month = int(data['month']['number'])
    year = int(data['year'])
    print(f"DEBUG: Hijri from API for {today} → {day} {month}/{year}")
    return day, month, year, now

def get_last_new_moon(date):
    t0 = ts.utc(date - timedelta(days=30))
    t1 = ts.utc(date + timedelta(days=1))
    times, phases = find_discrete(t0, t1, moon_phases(eph))
    new_moons = [t for t, p in zip(times, phases) if p == 0]
    print(f"DEBUG: Found {len(new_moons)} new moons in range {t0.utc_iso()} to {t1.utc_iso()}")
    return max(new_moons)

def check_visibility(city, dt):
    elevation = get_elevation(city['lat'], city['lon'])
    topos = wgs84.latlon(city['lat'], city['lon'], elevation)
    observer = earth + topos

    dt_start = datetime.combine(dt, datetime.min.time()).replace(tzinfo=utc)
    dt_end = datetime.combine(dt, datetime.max.time()).replace(tzinfo=utc)
    print(f"DEBUG: Visibility check window → {dt_start} to {dt_end}")

    f = sunrise_sunset(eph, topos)
    t0 = ts.utc(dt_start)
    t1 = ts.utc(dt_end)
    times, events = find_discrete(t0, t1, f)
    sunset = next((t for t, e in zip(times, events) if not e), None)

    if sunset is None:
        print("❌ Sunset time could not be determined.")
        return False, None

    print(f"DEBUG: Sunset time (UTC) → {sunset.utc_datetime()}")

    t_obs = ts.utc(sunset.utc_datetime() + timedelta(minutes=5))
    moon_ap = observer.at(t_obs).observe(moon).apparent()
    sun_ap = observer.at(t_obs).observe(sun).apparent()

    m_alt, _, _ = moon_ap.altaz()
    s_alt, _, _ = sun_ap.altaz()
    elongation = moon_ap.separation_from(sun_ap)

    print(f"\n📊 Crescent Check at {t_obs.utc_datetime().strftime('%Y-%m-%d %H:%M UTC')}")
    print(f"☀️ Sun Altitude: {s_alt.degrees:.2f}°")
    print(f"🌙 Moon Altitude: {m_alt.degrees:.2f}°")
    print(f"📏 Elongation  : {elongation.degrees:.2f}°")

    visible = m_alt.degrees > 5 and elongation.degrees > 8 and s_alt.degrees < -5
    return visible, sunset.utc_datetime().date()

def is_after_maghrib(city, date_time):
    elevation = get_elevation(city['lat'], city['lon'])
    topos = wgs84.latlon(city['lat'], city['lon'], elevation)
    observer = earth + topos

    date = date_time.date()
    dt_start = datetime.combine(date, datetime.min.time()).replace(tzinfo=utc)
    dt_end = datetime.combine(date, datetime.max.time()).replace(tzinfo=utc)

    f = sunrise_sunset(eph, topos)
    t0 = ts.utc(dt_start)
    t1 = ts.utc(dt_end)
    times, events = find_discrete(t0, t1, f)

    tz = pytz.timezone(city['tz'])
    sunset_time = next((t.utc_datetime().astimezone(tz) for t, e in zip(times, events) if not e), None)
    print(f"DEBUG: Sunset/Maghrib on {date} → {sunset_time}")

    return date_time >= sunset_time if sunset_time else False

# ========== MAIN ==========
def estimate_local_hijri(city):
    hijri_day, hijri_month, hijri_year, now = get_hijri_today(city)
    tz = pytz.timezone(city['tz'])
    today_date = now.date()
    api_start_date = today_date - timedelta(days=hijri_day - 1)

    print(f"\n📍 {city['name']} (TZ: {city['tz']})")
    print(f"🗓️ Gregorian Now: {now.strftime('%Y-%m-%d %H:%M:%S')} {city['tz']}")
    print(f"🧮 1st of this Hijri month (from API): {api_start_date}")

    new_moon = get_last_new_moon(api_start_date)
    visible, crescent_date = check_visibility(city, new_moon.utc_datetime().date())
    predicted_start = crescent_date + timedelta(days=2) if not visible else crescent_date + timedelta(days=1)

    # Get sunset for today
    after_maghrib = is_after_maghrib(city, now)

    # Real Islamic date is shifted only after today's maghrib
    hijri_predicted_day = (today_date - predicted_start).days + (1 if not after_maghrib else 2)
    real_predicted_date = predicted_start + timedelta(days=hijri_predicted_day - 1)

    print(f"🌑 New Moon: {new_moon.utc_datetime().date()} | Visible? {'✅' if visible else '❌'}")
    print(f"🌙 First Crescent Seen (Expected): {crescent_date} → 1st Hijri (Predicted): {predicted_start}")
    print(f"📅 Predicted Hijri Start Date: {predicted_start}")
    print(f"📅 API Hijri Start Date      : {api_start_date}")
    print(f"📅 Real-Time Gregorian Now   : {now.strftime('%Y-%m-%d %H:%M:%S')} {city['tz']}")

    if after_maghrib:
        print("🌓 It's after Maghrib: Islamic date has advanced.")
    else:
        print("🌞 It's before Maghrib: Islamic date remains the same.")

    print(f"🧮 Hijri date now (Predicted): {hijri_predicted_day} {hijri_month}/{hijri_year}")
    print(f"🕋 Hijri date (API — post-Maghrib logic): {hijri_day} {hijri_month}/{hijri_year}")

# ========== RUN ==========
city = get_city_data()
estimate_local_hijri(city)


DEBUG: Local datetime now → 2025-07-14 12:10:38.592244+05:00
DEBUG: Hijri from API for 2025-07-14 → 19 1/1447

📍 Islamabad (TZ: Asia/Karachi)
🗓️ Gregorian Now: 2025-07-14 12:10:38 Asia/Karachi
🧮 1st of this Hijri month (from API): 2025-06-26
DEBUG: Found 2 new moons in range 2025-05-27T00:00:00Z to 2025-06-27T00:00:00Z
DEBUG: Elevation for (33.6844, 73.0479) → 524.0m
DEBUG: Visibility check window → 2025-06-25 00:00:00+00:00 to 2025-06-25 23:59:59.999999+00:00
DEBUG: Sunset time (UTC) → 2025-06-25 14:22:03.944768+00:00

📊 Crescent Check at 2025-06-25 14:27 UTC
☀️ Sun Altitude: -1.74°
🌙 Moon Altitude: 2.08°
📏 Elongation  : 4.46°
DEBUG: Elevation for (33.6844, 73.0479) → 524.0m
DEBUG: Sunset/Maghrib on 2025-07-14 → 2025-07-14 19:19:20.659956+05:00
🌑 New Moon: 2025-06-25 | Visible? ❌
🌙 First Crescent Seen (Expected): 2025-06-25 → 1st Hijri (Predicted): 2025-06-27
📅 Predicted Hijri Start Date: 2025-06-27
📅 API Hijri Start Date      : 2025-06-26
📅 Real-Time Gregorian Now   : 2025-07-14 12:1